#  <center > Selling a house in Mexico City  </center>
##### <center> Alejandro Alfredo Morales Sánchez </center>

## Introduction

<b> Mexico City </b> is a one of the most representative cities in this country, where food, arquitecture and design are all in one place. Each neighborhoods has something to offer. In this notebook we will try to find the best possible location for a new restaurant. With a population of  8.855 million in 2015 and an area of 1,485 million kilometers squared is one of the most denses cities in the world. Mexico city is also a beatiful city for tourism where people are eager to know the culture through its famous dishes.

This report is specially targeted to stakeholders interested in opening a new restaurant in <b> Mexico City </b>. We would try to find the best location in terms of competition and income of neighborhoods.

In ordet to solve this problem we would use the <b> Foursquare API </b> to adquire the data and use data science techniques to clean it and analyze  it.

## Data


Given the nature of our problem we are intereste in:

<ul>
    <li> All cafeterias in a specific zone </li>
</ul>    


## Code

In [1]:
#Import libraries

import requests 
import pandas as pd 
import numpy as np 
from geopy.geocoders import Nominatim 
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
import folium 


## Download and clean data

In order to use the data we will use pgeocode a library "for high performance off-line querying of GPS coordinates, region name and municipality name from postal codes"

In [4]:
#Parameters

request_parameters = {
    "client_id": CLIENT_ID, 
    "client_secret": CLIENT_SECRET,
    "v": '20210301', #Format YYYYMMDD
    "section": "coffee", #Type of stablishment
    "near": "Mexico City", 
    "radius": 100000,
    "limit": 50}

#Request data

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)


In [5]:
#Data to json

d = data.json()["response"]

#Name of keys
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [6]:
#Locations

d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Mexico City', 'Mexico City')

In [7]:
#Limits

d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 19.495981200806472, 'lng': -99.07831137006355},
  'sw': {'lat': 19.339679199433984, 'lng': -99.21100724615988}},
 224)

In [8]:
#Geocode information usefel for folium

d["geocode"]

{'what': '',
 'where': 'mexico city',
 'center': {'lat': 19.42847, 'lng': -99.12766},
 'displayString': 'Mexico City, DF, Mexico',
 'cc': 'MX',
 'geometry': {'bounds': {'ne': {'lat': 19.515304989460464,
    'lng': -99.05579900650167},
   'sw': {'lat': 19.356858007471764, 'lng': -99.25983899084375}}},
 'slug': 'mexico-city',
 'longId': '72057594041458533'}

In [9]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [10]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [11]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 50


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '57d9af4c498eb1f22a68d455',
  'name': 'Tierra Garat',
  'location': {'address': 'Amberes 33',
   'crossStreet': 'Hamburgo',
   'lat': 19.425898829574308,
   'lng': -99.16557479536033,
   'labeledLatLngs': [{'label': 'display',
     'lat': 19.425898829574308,
     'lng': -99.16557479536033}],
   'cc': 'MX',
   'neighborhood': 'Juárez',
   'city': 'Ciudad de México',
   'state': 'Distrito Federal',
   'country': 'México',
   'formattedAddress': ['Amberes 33 (Hamburgo)',
    'Ciudad de México, Distrito Federal',
    'México']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups':

In [12]:
items[1]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '540673ad498e44dafcaed029',
  'name': 'Casa Tassel',
  'location': {'address': 'Córdoba 110',
   'lat': 19.418051,
   'lng': -99.158343,
   'labeledLatLngs': [{'label': 'display',
     'lat': 19.418051,
     'lng': -99.158343}],
   'postalCode': '06700',
   'cc': 'MX',
   'neighborhood': 'Roma Norte',
   'city': 'Ciudad de México',
   'state': 'Distrito Federal',
   'country': 'México',
   'formattedAddress': ['Córdoba 110',
    '06700 Ciudad de México, Distrito Federal',
    'México']},
  'categories': [{'id': '4bf58dd8d48988d1dc931735',
    'name': 'Tea Room',
    'pluralName': 'Tea Rooms',
    'shortName': 'Tea Room',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/tearoom_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []},
  'venuePage': {'id': '440206039'}},
 'refer

In [15]:
#Information for cafeterias

df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    #address = location["address"]
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("Found %i cafes" % len(df))

Found 41 cafes


In [16]:
#Dataframe info

df

,uid,name,shortname,address,postalcode,lat,lng
0,540673ad498e44dafcaed029,Casa Tassel,Tea Room,Álvaro Obregón 86,06700,19.418051,-99.158343
1,51ba6ba9498e9bcd16eee9ec,Cachito Mío Quiches & Tartas,Coffee Shop,Álvaro Obregón 86,06700,19.416528,-99.160931
2,56d76bac498ecdce6449901f,Tierra Garat,Coffee Shop,Álvaro Obregón 86,06700,19.418890,-99.161417
3,566c4847498e1bdb149c56b0,Qūentin Café,Café,Álvaro Obregón 86,06700,19.418648,-99.157651
4,57f56557498ef66f03d1da46,MARI'S PASTRY shop & taller,Coffee Shop,Álvaro Obregón 86,03020,19.394866,-99.159646
5,5b78d0bec9f907002c032d78,Tierra Garat,Coffee Shop,Álvaro Obregón 86,03100,19.388341,-99.171675
6,58421d7ef63c54143e54022f,Café Curado,Coffee Shop,Álvaro Obregón 86,06700,19.421470,-99.164129
7,5203c4da7dd20c1ba1ab8f13,Tomás - Casa Editora de Té,Tea Room,Álvaro Obregón 86,06100,19.411991,-99.173480
8,5ccda78616fa040039513084,Cafe Baveno,Café,Álvaro Obregón 86,06700,19.413537,-99.162881
9,58e2f685e03e574fc4fcfd28,Tierra Garat,Café,Álvaro Obregón 86,06100,19.410158,-99.172554


In [17]:
#Folium center

city_center = d["geocode"]["center"]
city_center

{'lat': 19.42847, 'lng': -99.12766}

In [22]:
import folium
import numpy
from folium import plugins

#map
map_mexico = folium.Map(location=[city_center["lat"], city_center["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_mexico)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_mexico.add_child(plugins.HeatMap(hm_data))

map_mexico

## Results

As we can see from the map above the western area of Mexico City is a highly competitive zone for cafeterias. This is due to the income of the neighborhoods, that this zones are highly attractive for tourists and the atmosphere. So a location near this places would be good but in not a high dense zone. 

## Conclusion

We can see that between Calle Tepic an Calle Tapachila there is not a single cafeteria, and there is a Hospital in this zone. This hospital represents a lot of potential clients. 

In [26]:
lat = 19.407716973314567
lng = -99.16533300857152
map_mexico = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Our New Cafe!",
    color='red',
    fill=True,
    fill_color='blue',
    fill_opacity=0.7,
    parse_html=False).add_to(map_mexico)
map_mexico